# Deliverable 2. Create a Customer Travel Destinations Map.
----
1. Create a folder called `Vacation_Search` to save all the files related with this deliverable.

2. Download the `Vacation_Search_starter_code.ipynb` Jupyter notebook,save it into your `Vacation_Search folder`, and rename it as `Vacation_Search.ipynb`.

3. In the `Vacation_Search.ipynb` file, ensure that the dependencies and the Google API key is imported correctly.

4. From the `Weather_Database` folder you created in the "Deliverable 1," import the `WeatherPy_Database.csv` file as a Pandas DataFrame named `city_data_df`.

5. Write two input statements that prompt the user to enter their minimum and maximum temperature criteria for their vacation.

6. Create a new Pandas DataFrame by using the `loc` Pandas method to filter the `city_data_df` DataFrame for temperature criteria  collected. Name the DataFrame as `preferred_cities_df`.

7. Create a new Pandas DataFrame named `clean_travel_cities` by using the Pandas `dropna` function on the `preferred_cities_df` to drop any empty rows.

8. Use the `copy` Pandas function to create a new DataFrame, called `hotel_df`, by copying the following columns from the `clean_travel_cities` DataFrame: "City", "Country", "Max Temp", "Current Description", "Lat", "Lng".

9. Add a new empty column named `Hotel Name` to the `hotel_df` DataFrame.

10. Review the hotel search parameters provided. These parameters are the same we used in this module; you'll use them to search for a hotel for each city.

11. Use a for loop to iterate through the `hotel_df` DataFrame, retrieve the latitude and longitude of each city to find the nearest hotel based on the search parameters provided, then add the hotel name to the `hotel_df` DataFrame. If a hotel isn't found, skip to the next city.

12. Drop any rows in the `hotel_df` DataFrame where a hotel name is not found and store the resulting data into a new DataFrame named `clean_hotel_df`.

13. Create an CSV file to store the `clean_hotel_df` DataFrame as `WeatherPy_vacation.csv` in the Vacation_Search folder.

14. Review the formatting template provided that you'll use to add an information box to each marker in the map. In the pop-up for each city you'll add:

    1. The city name.

    2. The country code.

    3. The weather description and maximum temperature for the city.

15. Review the provided list comprehension code to retrieve the city data from each row, which will then be added to the formatting template and saved in the `hotel_info` list.

16. Use the provided code snippet to retrieve the latitude and longitude from each row and store them in a new DataFrame called `locations`.

17. Refactor your previous marker layer map code to create a marker layer map that will have pop-up markers for each city on the map.

18. Take a screenshot of your map and save it to the `Vacation_Search` folder as `WeatherPy_vacation_map.png`.
---

## Ensure that the dependencies and the Google API key is imported correctly.

In [1]:
import sys
!{sys.executable} -m pip install citipy

In [2]:
!jupyter nbextension enable --py gmaps

Enabling notebook extension jupyter-gmaps/extension...
      - Validating: OK


In [3]:
!jupyter nbextension enable --py widgetsnbextension

Enabling notebook extension jupyter-js-widgets/extension...
      - Validating: OK


In [4]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import Google API key
from config import g_key

# Configure gmaps
gmaps.configure(api_key=g_key)

## From the `Weather_Database` folder you created in the "Deliverable 1," import the `WeatherPy_Database.csv` file as a Pandas DataFrame.

In [5]:
# Set the file path to import the WeatherPy_database.csv file
file_path = "../Weather_Database/weather_data_ch/cities1.csv"

# Load the CSV file into a Pandas DataFrame
city_data_df = pd.read_csv(file_path)

# Display sample data
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,City Description
0,0,Sinnamary,GF,5.3833,-52.9500,79.07,81,58,7.40,broken clouds
1,1,Hambantota,LK,6.1241,81.1185,79.14,94,40,0.00,moderate rain
2,2,Jamestown,US,42.0970,-79.2353,63.72,69,0,4.61,clear sky
3,3,Les Cayes,HT,18.2000,-73.7500,77.00,78,0,1.36,clear sky
4,4,Livingstonia,MW,-10.6060,34.1063,66.38,60,35,2.08,scattered clouds


In [6]:
! pwd && ls -al



/Users/noemimontelongo/Desktop/World_Weather_Analysis/Vacation_Search
total 288
drwxr-xr-x   7 noemimontelongo  staff     224 Oct 25 22:41 .
drwxr-xr-x  21 noemimontelongo  staff     672 Oct 25 22:21 ..
-rw-r--r--@  1 noemimontelongo  staff    6148 Oct 25 10:48 .DS_Store
drwxr-xr-x   3 noemimontelongo  staff      96 Oct 20 14:44 .ipynb_checkpoints
-rw-r--r--@  1 noemimontelongo  staff  132010 Oct 25 22:41 Vacation_Search.ipynb
drwxr-xr-x   4 noemimontelongo  staff     128 Oct 20 17:24 __pycache__
-rw-r--r--@  1 noemimontelongo  staff      70 Oct 20 16:49 config.py


## Write two input statements that prompt the user to enter their minimum and maximum temperature criteria for their vacation.

In [7]:
# Prompt the user to enter the minimum temperature criteria
min_temp = float(input("What is the minimum temperature you would like for your trip? "))

# Prompt the user to enter the maximum temperature criteria
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


## Create a new Pandas DataFrame by using the `loc` Pandas method to filter the `city_data_df` DataFrame for temperature criteria  collected.

In [8]:
# Filter the city_data_df DataFrame to find the cities that fit the criteria using the loc Pandas function
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
                                    
# Display sample data
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,City Description
0,0,Sinnamary,GF,5.3833,-52.9500,79.07,81,58,7.40,broken clouds
1,1,Hambantota,LK,6.1241,81.1185,79.14,94,40,0.00,moderate rain
3,3,Les Cayes,HT,18.2000,-73.7500,77.00,78,0,1.36,clear sky
26,26,Atuona,PF,-9.8000,-139.0333,77.31,73,6,17.65,clear sky
30,30,Kapaa,US,22.0752,-159.3190,87.78,79,40,12.66,scattered clouds
35,35,Nong Chik,TH,6.8436,101.1780,78.06,88,100,4.41,overcast clouds
36,36,Aden,YE,12.7794,45.0367,81.79,65,0,13.35,clear sky
45,45,Nabire,ID,-3.3667,135.4833,84.43,73,98,4.74,overcast clouds
50,50,Santa Isabel Do Rio Negro,BR,-0.4139,-65.0192,79.70,76,100,2.44,overcast clouds
51,51,Bathsheba,BB,13.2167,-59.5167,83.01,83,40,18.41,scattered clouds


## Create a new Pandas DataFrame named `clean_travel_cities` by using the Pandas `dropna` function on the `preferred_cities_df` to drop any empty rows.

In [9]:
# Drop any empty rows in the preferred_cities_df DataFrame and 
preferred_cities_df.isnull()



,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,City Description
0,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False
26,False,False,False,False,False,False,False,False,False,False
30,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...
679,False,False,False,False,False,False,False,False,False,False
682,False,False,False,False,False,False,False,False,False,False
687,False,False,False,False,False,False,False,False,False,False
690,False,False,False,False,False,False,False,False,False,False


In [10]:
preferred_cities_df.isnull().sum()

City_ID             0
City                0
Country             0
Lat                 0
Lng                 0
Max Temp            0
Humidity            0
Cloudiness          0
Wind Speed          0
City Description    0
dtype: int64

In [11]:
preferred_cities_df.dropna()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,City Description
0,0,Sinnamary,GF,5.3833,-52.9500,79.07,81,58,7.40,broken clouds
1,1,Hambantota,LK,6.1241,81.1185,79.14,94,40,0.00,moderate rain
3,3,Les Cayes,HT,18.2000,-73.7500,77.00,78,0,1.36,clear sky
26,26,Atuona,PF,-9.8000,-139.0333,77.31,73,6,17.65,clear sky
30,30,Kapaa,US,22.0752,-159.3190,87.78,79,40,12.66,scattered clouds
...,...,...,...,...,...,...,...,...,...,...
679,679,Garissa,KE,-0.4536,39.6401,75.58,84,88,8.39,overcast clouds
682,682,Pamanukan,ID,-6.2842,107.8106,77.20,86,100,3.98,overcast clouds
687,687,Mayor Pablo Lagerenza,PY,-19.9309,-60.7718,82.53,46,34,9.80,scattered clouds
690,690,Omboue,GA,-1.5746,9.2618,75.90,92,100,5.01,overcast clouds


In [12]:
#create a new DataFrame.
clean_travel_cities = pd.DataFrame(preferred_cities_df.dropna())

# Display sample data
clean_travel_cities

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,City Description
0,0,Sinnamary,GF,5.3833,-52.9500,79.07,81,58,7.40,broken clouds
1,1,Hambantota,LK,6.1241,81.1185,79.14,94,40,0.00,moderate rain
3,3,Les Cayes,HT,18.2000,-73.7500,77.00,78,0,1.36,clear sky
26,26,Atuona,PF,-9.8000,-139.0333,77.31,73,6,17.65,clear sky
30,30,Kapaa,US,22.0752,-159.3190,87.78,79,40,12.66,scattered clouds
...,...,...,...,...,...,...,...,...,...,...
679,679,Garissa,KE,-0.4536,39.6401,75.58,84,88,8.39,overcast clouds
682,682,Pamanukan,ID,-6.2842,107.8106,77.20,86,100,3.98,overcast clouds
687,687,Mayor Pablo Lagerenza,PY,-19.9309,-60.7718,82.53,46,34,9.80,scattered clouds
690,690,Omboue,GA,-1.5746,9.2618,75.90,92,100,5.01,overcast clouds


## Use the `copy` Pandas function to create a new DataFrame, called `hotel_df`, by copying the following columns from the `clean_travel_cities` DataFrame: "City", "Country", "Max Temp", "Current Description", "Lat", "Lng".

In [13]:
# Create DataFrame called hotel_df by copying some columns from the clean_travel_cities DataFrame.
hotel_df = clean_travel_cities[["City", "Country", "Max Temp", "Lat", "Lng","City Description"]].copy()

# Display sample data
hotel_df

,City,Country,Max Temp,Lat,Lng,City Description
0,Sinnamary,GF,79.07,5.3833,-52.9500,broken clouds
1,Hambantota,LK,79.14,6.1241,81.1185,moderate rain
3,Les Cayes,HT,77.00,18.2000,-73.7500,clear sky
26,Atuona,PF,77.31,-9.8000,-139.0333,clear sky
30,Kapaa,US,87.78,22.0752,-159.3190,scattered clouds
...,...,...,...,...,...,...
679,Garissa,KE,75.58,-0.4536,39.6401,overcast clouds
682,Pamanukan,ID,77.20,-6.2842,107.8106,overcast clouds
687,Mayor Pablo Lagerenza,PY,82.53,-19.9309,-60.7718,scattered clouds
690,Omboue,GA,75.90,-1.5746,9.2618,overcast clouds


## Add a new empty column named `Hotel Name` to the `hotel_df` DataFrame.

In [14]:
# Add a new empty column, "Hotel Name", to the hotel_df DataFrame
hotel_df = preferred_cities_df.dropna()[["City", "Country", "Max Temp", "Lat", "Lng","City Description"]].copy()
hotel_df["Hotel Name"] = ""


# Display sample data
hotel_df.head(10)

,City,Country,Max Temp,Lat,Lng,City Description,Hotel Name
0,Sinnamary,GF,79.07,5.3833,-52.9500,broken clouds,
1,Hambantota,LK,79.14,6.1241,81.1185,moderate rain,
3,Les Cayes,HT,77.00,18.2000,-73.7500,clear sky,
26,Atuona,PF,77.31,-9.8000,-139.0333,clear sky,
30,Kapaa,US,87.78,22.0752,-159.3190,scattered clouds,
35,Nong Chik,TH,78.06,6.8436,101.1780,overcast clouds,
36,Aden,YE,81.79,12.7794,45.0367,clear sky,
45,Nabire,ID,84.43,-3.3667,135.4833,overcast clouds,
50,Santa Isabel Do Rio Negro,BR,79.70,-0.4139,-65.0192,overcast clouds,
51,Bathsheba,BB,83.01,13.2167,-59.5167,scattered clouds,


## Review the hotel search parameters provided. These parameters are the same we used in this module; you'll use them to search for a hotel for each city.

In [15]:
# Review the parameters to search for a hotel
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}        

## Use a for loop to iterate through the `hotel_df` DataFrame, retrieve the latitude and longitude of each city to find the nearest hotel based on the search parameters provided, then add the hotel name to the `hotel_df` DataFrame. If a hotel isn't found, skip to the next city.

In [16]:
# Iterate through the hotel DataFrame 
for index, row in hotel_df.iterrows():
    # Fetch latitude and longitude from the DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    # Add the latitude and longitude as parameters to the params dictionary
    params["location"] = f"{lat},{lng}"
    
    # Set up the base URL for the Google Directions API to get JSON data
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # Make an API request and retrieve the JSON data from the hotel search
    hotels = hotels = requests.get(base_url, params=params).json()
    
    # Get the first hotel from the results and store the name, if a hotel isn't found skip the city
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [17]:
# Display sample data
hotel_df.head()

,City,Country,Max Temp,Lat,Lng,City Description,Hotel Name
0,Sinnamary,GF,79.07,5.3833,-52.9500,broken clouds,ALETHEIA Appartement Sinnamary
1,Hambantota,LK,79.14,6.1241,81.1185,moderate rain,Bungalow 63
3,Les Cayes,HT,77.00,18.2000,-73.7500,clear sky,Villa Orphee
26,Atuona,PF,77.31,-9.8000,-139.0333,clear sky,Villa Enata
30,Kapaa,US,87.78,22.0752,-159.3190,scattered clouds,Sheraton Kauai Coconut Beach Resort


## Drop any rows in the `hotel_df` DataFrame where a hotel name is not found and store the resulting data into a new DataFrame named `clean_hotel_df`.

In [18]:
hotel_df.isnull()


,City,Country,Max Temp,Lat,Lng,City Description,Hotel Name
0,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False
26,False,False,False,False,False,False,False
30,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...
679,False,False,False,False,False,False,False
682,False,False,False,False,False,False,False
687,False,False,False,False,False,False,False
690,False,False,False,False,False,False,False


In [19]:
# Drop the rows where there is no Hotel Name.
#clean_hotel_df = # YOUR CODE HERE
#df = df.drop(df[df.score < 50].index)

clean_hotel_df = hotel_df.drop(hotel_df[hotel_df["Hotel Name"] == True].index)


# Display sample data
# YOUR CODE HERE
hotel_df

,City,Country,Max Temp,Lat,Lng,City Description,Hotel Name
0,Sinnamary,GF,79.07,5.3833,-52.9500,broken clouds,ALETHEIA Appartement Sinnamary
1,Hambantota,LK,79.14,6.1241,81.1185,moderate rain,Bungalow 63
3,Les Cayes,HT,77.00,18.2000,-73.7500,clear sky,Villa Orphee
26,Atuona,PF,77.31,-9.8000,-139.0333,clear sky,Villa Enata
30,Kapaa,US,87.78,22.0752,-159.3190,scattered clouds,Sheraton Kauai Coconut Beach Resort
...,...,...,...,...,...,...,...
679,Garissa,KE,75.58,-0.4536,39.6401,overcast clouds,Sharif Plaza Hotel
682,Pamanukan,ID,77.20,-6.2842,107.8106,overcast clouds,Paris Day Hotel Pamanukan
687,Mayor Pablo Lagerenza,PY,82.53,-19.9309,-60.7718,scattered clouds,
690,Omboue,GA,75.90,-1.5746,9.2618,overcast clouds,Hotel Olako


In [20]:
# Create an CSV file to store the clean_hotel_df

**Note:** If the resulting DataFrame is empty, uncomment the following code to load sample data into the `clean_hotel_df`.

In [21]:
# In case of an empty DataFrame, load the sample data provided
# clean_hotel_df = pd.read_csv("WeatherPy_vacation-backup.csv")
# clean_hotel_df.head(10)

## Create an CSV file to store the `clean_hotel_df` DataFrame as `WeatherPy_vacation.csv` in the Vacation_Search folder.

In [22]:
# Set the file name.
output_data_file = "../WeatherPY_vacation.csv"


# Create a CSV file by using the clean_hotel_df DataFrame
hotel_df.to_csv(output_data_file, index_label="City_ID")

### Review the formatting template provided that you'll use to add an information box to each marker in the map.

In the pop-up for each city you'll add: 

1. The city name

2. The country code

3. The weather description and maximum temperature for the city

In [23]:
# Review the formatting template provided
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{City Description} and {Max Temp} °F</dd>
</dl>
"""

## Review the provided list comprehension code to retrieve the city data from each row, which will then be added to the formatting template and saved in the `hotel_info` list.

In [24]:
hotel_df

,City,Country,Max Temp,Lat,Lng,City Description,Hotel Name
0,Sinnamary,GF,79.07,5.3833,-52.9500,broken clouds,ALETHEIA Appartement Sinnamary
1,Hambantota,LK,79.14,6.1241,81.1185,moderate rain,Bungalow 63
3,Les Cayes,HT,77.00,18.2000,-73.7500,clear sky,Villa Orphee
26,Atuona,PF,77.31,-9.8000,-139.0333,clear sky,Villa Enata
30,Kapaa,US,87.78,22.0752,-159.3190,scattered clouds,Sheraton Kauai Coconut Beach Resort
...,...,...,...,...,...,...,...
679,Garissa,KE,75.58,-0.4536,39.6401,overcast clouds,Sharif Plaza Hotel
682,Pamanukan,ID,77.20,-6.2842,107.8106,overcast clouds,Paris Day Hotel Pamanukan
687,Mayor Pablo Lagerenza,PY,82.53,-19.9309,-60.7718,scattered clouds,
690,Omboue,GA,75.90,-1.5746,9.2618,overcast clouds,Hotel Olako


In [25]:
# Add a heatmap of temperature for the vacation spots and marker for each city.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,
             dissipating=False, max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [26]:
# Get the data from each row in the clean_hotel_df DataFrame, add it to the formatting template, and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
# hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
# Display sample data
hotel_info

['\n<dl>\n<dt>Hotel Name</dt><dd>ALETHEIA Appartement Sinnamary</dd>\n<dt>City</dt><dd>Sinnamary</dd>\n<dt>Country</dt><dd>GF</dd>\n<dt>Current Weather</dt><dd>broken clouds and 79.07 °F</dd>\n</dl>\n',
 '\n<dl>\n<dt>Hotel Name</dt><dd>Bungalow 63</dd>\n<dt>City</dt><dd>Hambantota</dd>\n<dt>Country</dt><dd>LK</dd>\n<dt>Current Weather</dt><dd>moderate rain and 79.14 °F</dd>\n</dl>\n',
 '\n<dl>\n<dt>Hotel Name</dt><dd>Villa Orphee</dd>\n<dt>City</dt><dd>Les Cayes</dd>\n<dt>Country</dt><dd>HT</dd>\n<dt>Current Weather</dt><dd>clear sky and 77.0 °F</dd>\n</dl>\n',
 '\n<dl>\n<dt>Hotel Name</dt><dd>Villa Enata</dd>\n<dt>City</dt><dd>Atuona</dd>\n<dt>Country</dt><dd>PF</dd>\n<dt>Current Weather</dt><dd>clear sky and 77.31 °F</dd>\n</dl>\n',
 '\n<dl>\n<dt>Hotel Name</dt><dd>Sheraton Kauai Coconut Beach Resort</dd>\n<dt>City</dt><dd>Kapaa</dd>\n<dt>Country</dt><dd>US</dd>\n<dt>Current Weather</dt><dd>scattered clouds and 87.78 °F</dd>\n</dl>\n',
 '\n<dl>\n<dt>Hotel Name</dt><dd>Bakkahland Farm

## Use the provided code snippet to retrieve the latitude and longitude from each row and store them in a new DataFrame called `locations`.

In [27]:
# Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df[["Lat", "Lng"]]

# Display sample data
locations.head(10)

,Lat,Lng
0,5.3833,-52.9500
1,6.1241,81.1185
3,18.2000,-73.7500
26,-9.8000,-139.0333
30,22.0752,-159.3190
35,6.8436,101.1780
36,12.7794,45.0367
45,-3.3667,135.4833
50,-0.4139,-65.0192
51,13.2167,-59.5167


## Refactor your previous marker layer map code to create a marker layer map that will have pop-up markers for each city on the map.

In [28]:
# Add a marker layer for each city to the map. 
# YOUR CODE HERE
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)


# Create a figure to add the Google map as a layer
# YOUR CODE HERE
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# Display the figure containing the map
# YOUR CODE HERE
fig

Figure(layout=FigureLayout(height='420px'))